In [1]:
%reset -f

import numpy as np
import matplotlib.pyplot as plt
from os import system

## Widgets
from ipywidgets import interact_manual, interactive_output
import ipywidgets as wd

## Visualization
from IPython.display import display, clear_output, Pretty

## PFLOTRAN
import jupypft.model as mo
import jupypft.parameter as pm
import jupypft.attachmentRateCFT as arCFT
import jupypft.plotBTC as plotBTC

In [2]:
templateFiles = ["../TEMPLATES/tpl_TH_3Dbox.in"]
#Pretty(templateFiles[0])

In [3]:
BoxModel = mo.Model(templateFile=templateFiles[0],
                       execPath="mpirun -np 4 $PFLOTRAN_DIR/buildExperimental/pflotran")

In [4]:
print(BoxModel)

mpirun -np 4 $PFLOTRAN_DIR/buildExperimental/pflotran -pflotranin ./pflotran.in


In [5]:
## Concentration
ConcentrationAtInlet = pm.Real("<initialConcentration>", value=1.00E-10, units="mol/L")

In [6]:
## Grid
nX,nY,nZ = pm.WithSlider("<nX>",units="-",mathRep="$$nX$$"),\
           pm.WithSlider("<nY>",units="-",mathRep="$$nY$$"),\
           pm.WithSlider("<nZ>",units="-",mathRep="$$nZ$$")

#nX.slider = wd.IntSlider(value=70,min=1,max=70,step=1)
#nY.slider = wd.IntSlider(value=30,min=1,max=70,step=1)
#nZ.slider = wd.IntSlider(value=20,min=1,max=70,step=1)

nX.slider = wd.IntSlider(value=6,min=1,max=70,step=1)
nY.slider = wd.IntSlider(value=4,min=1,max=70,step=1)
nZ.slider = wd.IntSlider(value=8,min=1,max=70,step=1)

ui_Grid = wd.VBox([nX.ui,nY.ui,nZ.ui])

In [7]:
## Dimensions
LX,LY,LZ = pm.WithSlider("<LenX>",units="m",mathRep="$$LX$$"),\
           pm.WithSlider("<LenY>",units="m",mathRep="$$LY$$"),\
           pm.WithSlider("<LenZ>",units="m",mathRep="$$LZ$$")

LX.slider = wd.FloatSlider(value=70, min=10, max=200, step=1)
LY.slider = wd.FloatSlider(value=30, min=10, max=200, step=1)
LZ.slider = wd.FloatSlider(value=20, min=10, max=200, step=1)

ui_Dimensions = wd.VBox([LX.ui,LY.ui,LZ.ui])

In [8]:
## General
LongDisp       = pm.WithSlider(tag="<longDisp>",units="m",mathRep="$$\\alpha_L$$")
RateDetachment = pm.WithSlider(tag="<kdet>",units="1/h",mathRep="$$k_{det}$$")
GradientX      = pm.WithSlider(tag="<GradientX>",units="1/h",mathRep="$$\partial_x h$$")

LongDisp.slider     = wd.FloatLogSlider(value=1.0E-12,base=10, min=-12, max=0, step=0.1)
GradientX.slider    = wd.FloatLogSlider(value=2.0E-3,base=10, min=-10, max=-2, step=0.1)
RateDetachment.slider = wd.FloatLogSlider(value=0.0026,base=10, min=-30, max=1, step=0.1)

ui_General = wd.VBox([LongDisp.ui, GradientX.ui, RateDetachment.ui])

In [9]:
## Attachment Rate (CFT)
AlphaEffic     = pm.WithSlider(tag="<alphaEfficiency>",units="-",mathRep="$$\\alpha$$")
GrainSize      = pm.WithSlider(tag="<diamCollector>",units="m",mathRep="$$d_c$$")
ParticleDiam   = pm.WithSlider(tag="<diamParticle>",units="m",mathRep="$$d_p$$")
HamakerConst   = pm.WithSlider(tag="<hamakerConstant>",units="J",mathRep="$$A$$")
ParticleDens   = pm.WithSlider(tag="<rhoParticle>",units="kg/m³",mathRep="$$\\rho_p$$")

## Attachment rates
GrainSize.slider    = wd.FloatLogSlider(value=2.0E-3,base=10, min=-4, max=-1, step=0.1)
ParticleDiam.slider = wd.FloatLogSlider(value=1.0E-7,base=10, min=-9, max=-4, step=0.1)
HamakerConst.slider = wd.FloatLogSlider(value=5.0E-21,base=10, min=-22, max=-18, step=0.1)
ParticleDens.slider = wd.FloatSlider(value=1050.0,min=1000.0, max=4000, step=100)
AlphaEffic.slider   = wd.FloatLogSlider(value=1E-5,base=10, min=-5, max=0, step=0.1)

ui_AttachmentRate = wd.VBox([GrainSize.ui,\
                                ParticleDiam.ui,\
                                HamakerConst.ui,\
                                ParticleDens.ui,\
                                AlphaEffic.ui])

In [10]:
## Permeability
Porosity = pm.WithSlider(tag="<porosity>",units="adim",mathRep="$$\\theta$$")
Porosity.slider = wd.FloatSlider(value=0.35, min=0.05, max=0.95, step=0.05)

kX,kY,kZ = pm.WithSlider(tag="<PermX>",units="m²",mathRep="$$k_{xx}$$"),\
           pm.WithSlider(tag="<PermY>",units="m²",mathRep="$$k_{yy}$$"),\
           pm.WithSlider(tag="<PermZ>",units="m²",mathRep="$$k_{zz}$$")

for k in [kX,kY,kZ]:
    k.slider = wd.FloatLogSlider(value=1.0E-8,base=10, min=-10, max=-7, step=0.5)
    
ui_Permeability = wd.VBox([kX.ui,kY.ui,kZ.ui,Porosity.ui])

In [11]:
## Temperatures
Tin, RefTemp = pm.WithSlider(tag="<injectTemp>",units="C",mathRep="$$T_{in}$$"),\
               pm.WithSlider(tag="<initialTemp>",units="C",mathRep="$$T_{0}$$"),\

Tin.slider     = wd.FloatSlider(value=10, min=1.0, max=35, step=1)
RefTemp.slider = wd.FloatSlider(value=10, min=1.0, max=35, step=1)

ui_Temperature = wd.VBox([Tin.ui,RefTemp.ui])

In [12]:
## Well locations and flowrates
inX,inY,inZ1,inZ2 = pm.WithSlider(tag="<inX>",units="m",mathRep="$$x_{Q_{in}}$$"),\
                    pm.WithSlider(tag="<inY>",units="m",mathRep="$$y_{Q_{in}}$$"),\
                    pm.WithSlider(tag="<inZ1>",units="m",mathRep="$$z_{1,Q_{in}}$$"),\
                    pm.WithSlider(tag="<inZ2>",units="m",mathRep="$$z_{2,Q_{in}}$$")

inX.slider = wd.FloatSlider(value=10, min=0.0, max=LX.value, step=0.5)
inY.slider = wd.FloatSlider(value=15, min=0.0, max=LY.value, step=0.5)
inZ1.slider = wd.FloatSlider(value=19, min=0.0, max=LZ.value, step=0.5)
inZ2.slider = wd.FloatSlider(value=20, min=0.0, max=LZ.value, step=0.5)

QinRate = pm.WithSlider(tag="<inRate>", units="m³/d",mathRep="$$Q_{in}$$")
QinRate.slider  = wd.FloatSlider(value=0.24, min=0., max=5., step=0.2)

ui_InjectionWell = wd.VBox([inX.ui,inY.ui,inZ1.ui,inZ2.ui,QinRate.ui])

In [13]:
outX,outY,outZ1,outZ2 = pm.WithSlider(tag="<outX>",units="m",mathRep="$$x_{Q_{out}}$$"),\
                        pm.WithSlider(tag="<outY>",units="m",mathRep="$$y_{Q_{out}}$$"),\
                        pm.WithSlider(tag="<outZ1>",units="m",mathRep="$$z_{1,Q_{out}}$$"),\
                        pm.WithSlider(tag="<outZ2>",units="m",mathRep="$$z_{2,Q_{out}}$$")

outX.slider = wd.FloatSlider(value=41., min=0.0, max=LX.value, step=0.5)
outY.slider = wd.FloatSlider(value=15., min=0.0, max=LY.value, step=0.5)
outZ1.slider = wd.FloatSlider(value=16., min=0.0, max=LZ.value, step=0.5)
outZ2.slider = wd.FloatSlider(value=19., min=0.0, max=LZ.value, step=0.5)

QoutRate = pm.WithSlider(tag="<outRate>",units="m³/d",mathRep="$$Q_{out}$$")
QoutRate.slider = wd.FloatSlider(value=-21.84, min=-30., max=0., step=0.2)

ui_ExtractionWell = wd.VBox([outX.ui,outY.ui,outZ1.ui,outZ2.ui,QoutRate.ui])

In [14]:
listOfParameters = pm.Parameter.list_of_vars()

In [15]:
def RunAll():
  
    #Copy the template as a runFile
    BoxModel.cloneTemplate()
    
    #Replace the tags in the runFile with values
    for parameter in listOfParameters:
        BoxModel.replaceTagInFile(parameter)
    
    #Run PFLOTRAN
    BoxModel.runModel()
    
    #Reformat the results file to a CSV
    BoxModel.fixedToCSV(outputFile="pflotran-mas.dat")
    BoxModel.fixedToCSV(outputFile="pflotran-obs-0.tec")
    
def plot1():
    clear_output()
    plotBTC.xyPlotLine(
        inputFile = "./pflotran-obs-0.tec",
        YIndex=17,
        normalizeXWith = 1.,
        normalizeYWith = ConcentrationAtInlet.value,
        legend = "BTC")

def plot2():
    clear_output()
    plotBTC.xyPlotLine(
        inputFile = "./pflotran-obs-0.tec",
        YIndex=8,
        normalizeXWith = 1.,
        normalizeYWith = ConcentrationAtInlet.value,
        legend = "BTC")

In [16]:
ui_RunModel = wd.Button(
    description='Run PFLOTRAN',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run PFLOTRAN',
    icon='check'
)

ui1 = wd.Accordion(children=[ui_General,\
                             ui_Grid,\
                             ui_Dimensions,\
                             ui_Permeability,\
                             ui_Temperature,\
                             ui_InjectionWell,\
                             ui_ExtractionWell,\
                             ui_AttachmentRate,\
                             ui_RunModel
                            ])
ui1.set_title(0,"General")
ui1.set_title(1,"Grid")
ui1.set_title(2,"Dimensions")
ui1.set_title(3,"Permeability")
ui1.set_title(4,"Temperature")
ui1.set_title(5,"InjectionWell")
ui1.set_title(6,"ExtractionWell")
ui1.set_title(7,"Attachment rate (CFT)")
ui1.set_title(8,"Run Model")

In [17]:
ui2_1 = wd.Output()
ui2_2 = wd.Output()

def on_button_clicked(_):
    RunAll()
    with ui2_1: plot1()
    with ui2_2: plot2()

ui_RunModel.on_click(on_button_clicked)

ui2 = wd.Tab(children=[ui2_1,ui2_2])
ui2.set_title(0,"Extract Well probe")
ui2.set_title(1,"Injection Well probe")

In [18]:
ui = wd.TwoByTwoLayout(top_left=ui1,
                  top_right=ui2)
display(ui)

TwoByTwoLayout(children=(Accordion(children=(VBox(children=(HBox(children=(HTMLMath(value='$$\\alpha_L$$'), Fl…

In [19]:
Pretty("./pflotran.in")

#==============================================================================
#Description: 3D groundwater flow and conservative transport. 
#Template: one layer, well 40 m away, not fully sat.
#Flow: RICHARDS
#==============================================================================

SIMULATION
  SIMULATION_TYPE SUBSURFACE
  PROCESS_MODELS
    SUBSURFACE_FLOW flow
      MODE TH
    /
    SUBSURFACE_TRANSPORT transport
      MODE GIRT
    /
  /
END

SUBSURFACE

#=========================== numerical methods ================================

NUMERICAL_METHODS FLOW
  NEWTON_SOLVER
    ITOL_UPDATE 1.d0     ! Convergences with max change in pressure is 1 Pa.
  /
END

NUMERICAL_METHODS transport
  TIMESTEPPER
    CFL_GOVERNOR 1.0
  /
END

#=========================== chemistry ========================================
CHEMISTRY
  PRIMARY_SPECIES
    Vaq
  /
  
  IMMOBILE_SPECIES
    Vim
  /

  REACTION_SANDBOX
    BIOPARTICLE
      PARTICLE_NAME_AQ Vaq
      PARTICLE_NAME_IM Vim
     